In [1]:
import numpy as np
import scipy.io as sio
import pickle
import os

In [2]:
# Load the Schools dataset
schools_name = "data/school/schoolData.mat"
schools_dataset = sio.loadmat(schools_name)
print(schools_dataset.keys())
X_schools = schools_dataset['X']
X_schools = np.squeeze(X_schools)
Y_schools = schools_dataset['Y']
Y_schools = np.squeeze(Y_schools)

#for i in range(X_schools.shape[0]):
#    print(i, end=' ')
#    print(X_schools[i].shape[1])
#task 71 becomes the test task

T = X_schools.shape[0]
d1 = X_schools[0].shape[0]
d2 = 50
print("T = {}".format(T))
print("d1 = {}".format(d1))

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Y'])
T = 139
d1 = 26


In [3]:
# prepare test task
def transform_Y(X,d2):
    sigma = 10

    # Generate the parameters for the transform
    sum_phi = np.zeros(d2)
    v = np.random.uniform(0, 2 * np.pi, size=d2)
    U = np.random.normal(0, sigma, size=(d2, X.shape[1]))
    for i in range(X.shape[0]):
        sum_phi += np.sqrt(2/d2) * np.cos(U @ X[i] + v)
    avg_phi = np.divide(sum_phi, X.shape[0])
    return avg_phi

# we pick task 29 as the test task as it has enough samples to vary N2 
task = 29
test_task = X_schools[task]
test_labels = Y_schools[task]
X0 = np.transpose(test_task)
R0 = test_labels
Y0 = transform_Y(X0,d2)
#normalize test task
normx0 = np.linalg.norm(X0, axis=1).reshape(len(X0),1)
#
X0new = X0/normx0
R0new = R0/normx0
# save test task
pickle.dump(X0new, open('./data/school/X0.pkl', "wb"))
pickle.dump(Y0, open('./data/school/Y0.pkl', "wb"))
pickle.dump(R0new, open('./data/school/R0.pkl', "wb"))
# 

In [4]:
# prepare training dataset

# select 50 tasks uniformly at random except task 29
indices = np.random.randint(0,T-1,size=50)
v = np.arange(139)
v = np.delete(v,29)
train_tasks = v[indices]

Ytrain = np.zeros((50,d2))
for i in range(50):
    X = np.transpose(X_schools[train_tasks[i]])
    Ytrain[i,] = transform_Y(X,d2)
    
#20 samples from each task in train_tasks
Xtrain = np.zeros((50*20, d1))
task_function = np.zeros(50*20)
Rtrain = np.zeros((50*20,1))

for i in range(50):
    X = np.transpose(X_schools[train_tasks[i]])
    indices = np.random.randint(0,X.shape[0],size=20)
    Xtrain[(20*i):(20*i + 20),] = X[indices,]
    task_function[(20*i):(20*i + 20)] = i
    Rtrain[(20*i):(20*i + 20)] = Y_schools[train_tasks[i]][indices] 
    
#normalize data
normXt = np.linalg.norm(Xtrain, axis=1).reshape(len(Xtrain),1)
Xtrain_new = Xtrain/normXt
Rtrain_new = Rtrain/normXt

# save meta-train dataset
pickle.dump(Xtrain_new, open('./data/school/X.pkl', "wb"))
pickle.dump(Ytrain, open('./data/school/Y.pkl', "wb"))
pickle.dump(Rtrain_new, open('./data/school/R.pkl', "wb"))
pickle.dump(task_function, open('./data/school/task_function.pkl', "wb"))

#create an evaluation dataset
Xtrain_eval = np.zeros((50*20, d1))
task_function_eval = np.zeros(50*20)
Rtrain_eval = np.zeros((50*20,1))

for i in range(50):
    X = np.transpose(X_schools[train_tasks[i]])
    indices = np.random.randint(0,X.shape[0],size=20)
    Xtrain_eval[(20*i):(20*i + 20),] = X[indices,]
    task_function_eval[(20*i):(20*i + 20)] = i
    Rtrain_eval[(20*i):(20*i + 20)] = Y_schools[train_tasks[i]][indices] 
    
#normalize data
normXt = np.linalg.norm(Xtrain_eval, axis=1).reshape(len(Xtrain_eval),1)
Xtrain_new = Xtrain_eval/normXt
Rtrain_new = Rtrain_eval/normXt

# save meta-train dataset
pickle.dump(Xtrain_new, open('./data/school/Xtrain_eval.pkl', "wb"))
pickle.dump(Ytrain, open('./data/school/Ytrain_eval.pkl', "wb"))
pickle.dump(Rtrain_new, open('./data/school/Rtrain_eval.pkl', "wb"))
pickle.dump(task_function, open('./data/school/task_function_eval.pkl', "wb"))

In [5]:
#create an evaluation set for the meta-test task

indices = np.random.randint(0,len(X0new),size=50)
X0eval = X0new[indices,]
R0eval = R0new[indices,]

pickle.dump(X0eval, open('./data/school/X0eval.pkl', "wb"))
pickle.dump(R0eval, open('./data/school/R0eval.pkl', "wb"))

#generate test data for 20 different trials and different sizes
X0test = X0new[-indices,]
R0test = R0new[-indices,]

for trial in range(20):
    for s in range(20,200,20):
        indices = np.random.randint(0, len(X0test), size=s)
        cX0 = X0test[indices,]
        cR0 = R0test[indices,]
        pickle.dump(cX0, open('./data/school/' + 'X0_N2_{N2F}_trial_{trialF}.pkl'.format(N2F=s, trialF=trial), 'wb'))
        pickle.dump(cR0, open('./data/school/' + 'R0_N2_{N2F}_trial_{trialF}.pkl'.format(N2F=s, trialF=trial), 'wb'))
        